# Classic Radiosity with Python and Mitsuba 3
This notebook demonstrates a classic implementation of the radiosity algorithm. The process involves:
1. Mesh Refinement: Subdividing large triangles in a scene into smaller patches to ensure accurate light distribution.
2. Form Factor Calculation: Computing a matrix F where $F_{ij}$ represents the fraction of energy leaving patch i that arrives at patch j. This is the most computationally intensive step.
3. Radiosity Solver: Iteratively solving the radiosity equation to find the final radiosity (exitant light) of each patch.
4. Rendering: Using Mitsuba 3 to render the final scene, where the calculated radiosity values are used as per-face emission colors.
## 1. Setup and Imports

In [1]:
import os
import glob
from math import ceil, sqrt
from collections import defaultdict
from multiprocessing import Pool

import numpy as np
from numpy.linalg import norm
from scipy.spatial import Delaunay, cKDTree
from scipy.spatial.distance import pdist
from scipy.sparse import csr_matrix

import matplotlib.pyplot as plt
import pyvista as pv
from tqdm.auto import tqdm

import trimesh
from trimesh.ray.ray_pyembree import RayMeshIntersector
from plyfile import PlyData, PlyElement

import mitsuba as mi

temp_dir = "./temp"
mi.set_variant("cuda_ad_rgb")
os.makedirs(temp_dir, exist_ok=True)

# Utility Funcitons

In [2]:
def visualize_triangles(triangles, triangle_values=None, cmap='magma'):
    vertices = triangles.reshape(-1, 3)
    
    # Create face array: [3, pt1, pt2, pt3, 3, pt4, pt5, pt6, ...]
    faces = np.hstack([[3] + list(f) for f in np.arange(len(vertices)).reshape(-1, 3)])
    
    mesh = pv.PolyData(vertices, faces)
    
    # Add scalar values as cell data (one value per triangle)
    if triangle_values is not None:
        mesh.cell_data["values"] = triangle_values
    
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, scalars="values" if triangle_values is not None else None, show_edges=True, opacity=0.9, cmap=cmap, color='skyblue')
    plotter.show()

In [3]:
def plotPointsViaDelaunayTriangulation(pnts, filename=''):
    points = np.array(pnts)
    tri = Delaunay(points)
    
    plt.figure(dpi=300)
    plt.triplot(points[:,0], points[:,1], tri.simplices.copy(), linewidth=0.5)
    plt.plot(points[:,0], points[:,1], 'o', ms=3)
    
    if filename: 
        plt.savefig(filename)
    else:
        plt.show()

## Refining Triangles

In [4]:
def project_triangle_2D(triangle):
    v0, v1, v2 = triangle
    
    origin = v0

    u = v1 - v0
    u /= np.linalg.norm(u)
    
    normal = np.cross(v1 - v0, v2 - v0)
    normal /= np.linalg.norm(normal)
    
    v = np.cross(normal, u)
    
    def project_to_2d(p):
        relative = p - origin
        return [np.dot(relative, u), np.dot(relative, v)]
    
    v0_2d = project_to_2d(v0)  # will be (0, 0)
    v1_2d = project_to_2d(v1)
    v2_2d = project_to_2d(v2)

    return v0_2d, v1_2d, v2_2d, origin, u, v

In [5]:
def splitViaDelaunay(points, maxLength):
    # print("Perform Delaunay triangulation with "+str(len(points))+" points")
    tri = Delaunay(points)
    # plotPointsViaDelaunayTriangulation(points, filename=f'./triangle_{len(points)}.png')
    # get set of edges from the simpleces
    edges = set()
    for simplex in tri.simplices:
        # simplex is one triangle: [ 4  5 17]
        edges.add((simplex[0], simplex[1]))
        edges.add((simplex[1], simplex[2]))
        edges.add((simplex[0], simplex[2]))
    # check if all edges are small enough
    # and add new points if not
    isFinished = True
    for edge in edges:
        p1, p2 = edge
        [x1, y1] = points[p1]
        [x2, y2] = points[p2]
        length = sqrt((x2-x1)*(x2-x1)+(y2-y1)*(y2-y1))
        if length > maxLength:
            isFinished = False
            # split in how many pieces?
            nPieces = ceil(length/maxLength)
            for piece in range(1, int(nPieces)):
                points.append([x1+piece/float(nPieces)*(x2-x1), y1+piece/float(nPieces)*(y2-y1)])
    # plotPointsViaDelaunayTriangulation(points)
    if not isFinished:
        splitViaDelaunay(points, maxLength)

In [6]:
def subdivide_triangles(triangles, maxLength = 0.5):
    refined_triangles = []
    
    for triangle in tqdm(triangles, desc="Refinig Triangles", leave=False):
        v0_2d, v1_2d, v2_2d, origin, u, v = project_triangle_2D(triangle)
        points = [v0_2d, v1_2d, v2_2d]
        
        splitViaDelaunay(points, maxLength)  
        tri = Delaunay(points)
    
        def to_3d(p, origin, u, v):
            x, y = p
            return origin + u * x + v * y
            
        for x, y, z in tri.simplices:
            v0, v1, v2 = points[x], points[y], points[z]
            v0, v1, v2 = to_3d(v0, origin, u, v), to_3d(v1, origin, u, v), to_3d(v2, origin, u, v)
            refined_triangles.append([v0, v1, v2])
    
    return refined_triangles

In [7]:
def refine_scene(scene, maxLength=0.5):
    refined_scene = trimesh.Scene()
    
    for key, mesh in scene.geometry.items():
        triangles = mesh.triangles  # (N, 3, 3)
    
        refined_triangles = subdivide_triangles(triangles, maxLength)
    
        # Build new vertices and faces
        vertices = np.array(refined_triangles).reshape(-1, 3)
        faces = np.arange(len(vertices)).reshape(-1, 3)
    
        # Preserve visual/material if present
        mat = mesh.visual.material if hasattr(mesh.visual, 'material') else None
        visual = trimesh.visual.TextureVisuals(material=mat) if mat else None
    
        new_mesh = trimesh.Trimesh(vertices=vertices, faces=faces, process=False, visual=visual)
        new_mesh.fix_normals()
        new_mesh.name = key
        refined_scene.add_geometry(new_mesh, node_name=key)
    
    return refined_scene

# Code

In [8]:
filename = "CornellBox-Sphere.obj"
scene = trimesh.load("./scenes/" + filename, force='scene')  # 'force=scene' is important if multiple geometries exist
triangles = scene.triangles

scene.geometry.keys()

odict_keys(['light', 'leftWall', 'rightWall', 'backWall', 'ceiling', 'floor', 'rightSphere', 'leftSphere'])

In [9]:
visualize_triangles(scene.triangles)

Widget(value='<iframe src="http://localhost:57219/index.html?ui=P_0x18416d6d3d0_0&reconnect=auto" class="pyvis…

In [29]:
refined_scene = refine_scene(scene, maxLength=0.1)
refined_scene.export("refined_scene.obj")
print("Exported Refined object File")

Refinig Triangles:   0%|          | 0/2 [00:00<?, ?it/s]

Refinig Triangles:   0%|          | 0/2 [00:00<?, ?it/s]

Refinig Triangles:   0%|          | 0/2 [00:00<?, ?it/s]

Refinig Triangles:   0%|          | 0/2 [00:00<?, ?it/s]

Refinig Triangles:   0%|          | 0/2 [00:00<?, ?it/s]

Refinig Triangles:   0%|          | 0/2 [00:00<?, ?it/s]

Refinig Triangles:   0%|          | 0/1088 [00:00<?, ?it/s]

Refinig Triangles:   0%|          | 0/1088 [00:00<?, ?it/s]

Exported Refined object File


In [30]:
# colors = np.zeros(refined_scene.triangles.shape)
# colors[0] = 3333
visualize_triangles(refined_scene.triangles)

Widget(value='<iframe src="http://localhost:57219/index.html?ui=P_0x1843be68410_3&reconnect=auto" class="pyvis…

## Triangulation

In [31]:
%matplotlib inline

points = [[0,0], [10,3], [9.5,4]]
# plotPointsViaDelaunayTriangulation(points)
splitViaDelaunay(points, 0.5)
# plotPointsViaDelaunayTriangulation(points)

## 

In [32]:
T = [] # traingle Vertices
rho = []  # Reflectance
E = []    # Emission

for key, mesh in tqdm(refined_scene.geometry.items(), 'Extracting triangles'):
    N = len(mesh.triangles)
    T.extend(mesh.triangles)

    props = mesh.visual.material.kwargs

    kd = np.array(props.get('kd', [0.5, 0.5, 0.5]), dtype=np.float32)
    ke = np.array(props.get('ke', [0.0, 0.0, 0.0]), dtype=np.float32)

    rho.extend([kd] * N)
    E.extend([ke] * N)

Extracting triangles:   0%|          | 0/8 [00:00<?, ?it/s]

In [33]:
T = np.array(T)
rho = np.array(rho)
E = np.array(E)
N = len(T)

print(f"Total patches: {N}")
print(f"Shapes: T={T.shape}, rho={rho.shape}, E={E.shape}")

Total patches: 11254
Shapes: T=(11254, 3, 3), rho=(11254, 3), E=(11254, 3)


In [34]:
all_vertices = T.reshape(-1, 3)
all_faces = np.arange(len(all_vertices)).reshape(-1, 3)
scene_mesh = trimesh.Trimesh(vertices=all_vertices, faces=all_faces, process=False)

## 5. Form Factor Calculation

The **form factor** $F_{ij}$ represents the fraction of energy leaving patch $i$ that directly reaches patch $j$. It is a key component in **radiosity** calculations.

We use a **point-to-patch** approximation:

$$
F_{ij} \approx \frac{\cos(\theta_i)\cos(\theta_j)}{\pi r^2} A_j \cdot V_{ij}
$$

---

### Where:

- $\theta_i$: Angle between the normal of patch $i$ and the line connecting the centroids of patches $i$ and $j$
- $\theta_j$: Angle between the normal of patch $j$ and the same connecting line  
- $r$: Distance between the centroids of patches $i$ and $j$  
- $A_j$: Area of patch $j$  
- $V_{ij}$: **Visibility term**  
  - $1$ if patch $j$ is visible from patch $i$  
  - $0$ otherwise

---

This approximation assumes that each patch can be represented by its centroid and normal, which simplifies computation while maintaining reasonable accuracy.

In [35]:
N = len(T)
rho = np.array(rho)
E = np.array(E)

centroids = scene_mesh.triangles_center
normals = scene_mesh.face_normals
areas = scene_mesh.area_faces

F = np.zeros((N, N))
V = np.zeros((N, N), dtype=np.uint8)

scene_rays = RayMeshIntersector(scene_mesh)

# Build KD-tree on triangle centroids
centroid_tree = cKDTree(centroids)

bbox_min = np.min(centroids, axis=0)
bbox_max = np.max(centroids, axis=0)

max_neighbor_radius = norm(bbox_max - bbox_min)  # diagonal of bounding box

epsilon = 1e-5 * np.linalg.norm(bbox_max - bbox_min)

In [36]:
# Check if form factor is cached
F_path = f"./temp/{filename}_{N}_F.npy"

if os.path.exists(F_path):
    print("Loading existing form factor and visibility matrices...")
    F = np.load(F_path)
else:
    print("Computing form factor and visibility matrices...")
    
    for i in tqdm(range(N), desc="Calculating Form Factors"):
        origin_i = centroids[i]
        normal_i = normals[i]
        
        # Query all other patches (potential neighbors)
        # Using the full scene bounding box diagonal is a safe but loose upper bound for neighbors
        potential_j_indices = list(range(N))
        potential_j_indices.remove(i)
        
        if not potential_j_indices:
            continue
            
        # Vectorized calculations for all potential neighbors
        vecs_ij = centroids[potential_j_indices] - origin_i
        distances_ij = np.linalg.norm(vecs_ij, axis=1)
        directions_ij = vecs_ij / distances_ij[:, np.newaxis]
        
        # Calculate cosine terms
        cos_theta_i = directions_ij @ normal_i
        
        # Note the negative sign: cos_theta_j is dot(direction_ij, normal_j), but the form factor
        # uses the angle with the incoming ray, so we use -direction_ij.
        cos_theta_j = -np.einsum('ij,ij->i', directions_ij, normals[potential_j_indices])
        
        # Back-face culling: only consider patches that are facing each other
        valid_mask = (cos_theta_i > epsilon) & (cos_theta_j > epsilon)
        
        if not np.any(valid_mask):
            continue
            
        # Filter to only potentially visible patches
        j_indices = np.array(potential_j_indices)[valid_mask]
        dirs = directions_ij[valid_mask]
        dists = distances_ij[valid_mask]
        
        # Ray origins are slightly offset from the surface to avoid self-intersection
        ray_origins = origin_i + epsilon * dirs
        
        # Perform batched ray-casting to check for occluders
        # We only need the first hit for each ray.
        hit_locations, index_ray, index_tri = scene_rays.intersects_location(
            ray_origins=ray_origins,
            ray_directions=dirs,
            multiple_hits=False
        )
        
        # Assume all are visible initially
        is_visible = np.ones(len(j_indices), dtype=bool)
        
        if len(hit_locations) > 0:
            # Calculate distance to the first object hit by each ray
            hit_dists = np.linalg.norm(hit_locations - ray_origins[index_ray], axis=1)
            
            # If the hit distance is less than the distance to the target patch j, it's occluded.
            expected_dists = dists[index_ray]
            blocked = hit_dists < expected_dists - epsilon
            
            # Update visibility for the rays that were blocked
            is_visible[index_ray[blocked]] = False
        
        # Final filtering for visible patches
        visible_j_indices = j_indices[is_visible]
        if len(visible_j_indices) == 0:
            continue
        
        # Get geometric terms for only the visible patches
        visible_cos_i = cos_theta_i[valid_mask][is_visible]
        visible_cos_j = cos_theta_j[valid_mask][is_visible]
        visible_dists = dists[is_visible]
        
        # Calculate and store the form factor value
        form_factor_value = (visible_cos_i * visible_cos_j * areas[visible_j_indices]) / (np.pi * visible_dists**2)
        F[i, visible_j_indices] = np.clip(form_factor_value, 0, 1)


        # Save after computing
    np.save(F_path, F)
    print("Saved form factor and visibility matrices.")

F_sparse = csr_matrix(F)

Computing form factor and visibility matrices...


Calculating Form Factors:   0%|          | 0/11254 [00:00<?, ?it/s]

Saved form factor and visibility matrices.


In [37]:
visualize_triangles(T, triangle_values=F)

Widget(value='<iframe src="http://localhost:57219/index.html?ui=P_0x18473872ad0_4&reconnect=auto" class="pyvis…

In [38]:
def create_ply_with_vertex_color(triangles, vertex_colors, output_filename, binary=True):
    # Flattened list of all vertices (duplicated per face)
    all_vertices = triangles.reshape(-1, 3)
    all_faces = np.arange(len(all_vertices)).reshape(-1, 3)
    triangle_colors = vertex_colors
    
    # Get unique vertices
    unique_vertices, inverse_indices = np.unique(all_vertices, axis=0, return_inverse=True)
    num_unique = unique_vertices.shape[0]
    
    # Build per-vertex color from triangle colors
    vertex_colors = np.zeros((num_unique, 3), dtype=np.float64)
    vertex_counts = np.zeros(num_unique, dtype=np.int32)
    
    # Loop over all triangles
    for tri_idx, face in enumerate(all_faces):
        for local_vidx in range(3):  # 3 vertices per triangle
            global_vidx = inverse_indices[tri_idx * 3 + local_vidx]
            vertex_colors[global_vidx] += triangle_colors[tri_idx]
            vertex_counts[global_vidx] += 1
    
    # Average the colors
    vertex_colors /= vertex_counts[:, None]

    # Pack into structured array
    vertex_dtype = np.dtype([
        ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
        ('red', 'f4'), ('green', 'f4'), ('blue', 'f4')
    ])
    vertex_data = np.zeros(len(unique_vertices), dtype=vertex_dtype)
    vertex_data['x'] = unique_vertices[:, 0]
    vertex_data['y'] = unique_vertices[:, 1]
    vertex_data['z'] = unique_vertices[:, 2]
    vertex_data['red'] = vertex_colors[:, 0]
    vertex_data['green'] = vertex_colors[:, 1]
    vertex_data['blue'] = vertex_colors[:, 2]
    
    remapped_faces = inverse_indices.reshape(-1, 3)  # shape: (n_faces, 3)
    # Face data
    face_dtype = np.dtype([('vertex_indices', 'i4', (3,))])
    face_data = np.array([(face,) for face in remapped_faces], dtype=face_dtype)
    
    # Write PLY
    ply = PlyData([
        PlyElement.describe(vertex_data, 'vertex'),
        PlyElement.describe(face_data, 'face')
    ], text=False)
    
    ply.write(output_filename)

In [39]:
def create_ply_with_face_color(triangles, face_colors, output_filename):
    """
    Creates a PLY file where each face has a specified color.
    Each triangle gets a uniform color (face color).
    """
    all_vertices = triangles.reshape(-1, 3)
    all_faces = np.arange(len(all_vertices)).reshape(-1, 3)
    triangle_colors = face_colors
    
    # Duplicate vertices per triangle
    duplicated_vertices = all_vertices[all_faces].reshape(-1, 3)  # shape: (n_faces * 3, 3)
    
    # Create new face indices
    n_faces = all_faces.shape[0]
    duplicated_faces = np.arange(n_faces * 3).reshape(n_faces, 3)
    
    # Assign triangle color to each vertex
    # Each triangle contributes its color to 3 vertices
    per_vertex_colors = np.repeat(triangle_colors, 3, axis=0)  # shape: (n_faces * 3, 3)
    
    # Pack into structured array for PLY
    vertex_dtype = np.dtype([
        ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
        ('red', 'f4'), ('green', 'f4'), ('blue', 'f4')
    ])
    vertex_data = np.zeros(len(duplicated_vertices), dtype=vertex_dtype)
    vertex_data['x'] = duplicated_vertices[:, 0]
    vertex_data['y'] = duplicated_vertices[:, 1]
    vertex_data['z'] = duplicated_vertices[:, 2]
    vertex_data['red'] = per_vertex_colors[:, 0]
    vertex_data['green'] = per_vertex_colors[:, 1]
    vertex_data['blue'] = per_vertex_colors[:, 2]
    
    # Face data
    face_dtype = np.dtype([('vertex_indices', 'i4', (3,))])
    face_data = np.array([(f,) for f in duplicated_faces], dtype=face_dtype)
    
    # Write binary PLY
    ply = PlyData([
        PlyElement.describe(vertex_data, 'vertex'),
        PlyElement.describe(face_data, 'face')
    ], text=False)  # text=False → binary format (recommended)
    
    ply.write(output_filename)

    # print(f"Created PLY with face colors at '{output_filename}'")

In [40]:
def render_with_mitsuba(triangles, radiosity_colors, spp=128, smooth=False):
    """
    Renders the scene using the calculated radiosity values.
    """
    yaw= 0.0
    pitch= -270.0
    radius= 10 # 10 for cbox, 4.5 for sphere cbox
    
    # Convert angles to radians
    yaw_rad = np.radians(yaw)
    pitch_rad = np.radians(pitch)
    
    # Spherical to Cartesian for camera position
    x = radius * np.cos(pitch_rad) * np.sin(yaw_rad)
    y = radius * np.cos(pitch_rad) * np.cos(yaw_rad)
    z = radius * np.sin(pitch_rad)
    
    origin = [x, y + 1, z] # y + 1 for others
    target = [0, 2.5, 0] # 2.5 for cbox, 1 for sphere cbox
    up = [0, 1, 0]
    
    ply_filename = "radiosityresult.ply"
    if smooth:
        create_ply_with_vertex_color(triangles, radiosity_colors, ply_filename)   
    else:
        create_ply_with_face_color(triangles, radiosity_colors, ply_filename)   
          
    # Define the scene for Mitsuba
    scene_dict = {
        "type": "scene",
        "integrator": {"type": "path", "max_depth": 1},
        
        "sensor": {
            "type": "perspective",
            "to_world": mi.ScalarTransform4f.look_at(origin, target, up),
            "film": {
                "type": "hdrfilm",
                "width": 1600,
                "height": 1600  
            },
    
            "sampler": {
                "type": "independent",
                "sample_count": spp  # Number of samples per pixel
            }
        },
        
        # The main object is a PLY mesh acting as an emitter
        "radiosity_mesh": {
            "type": "ply",
            "filename": ply_filename,
            "emitter": {
                "type": "area",
                "radiance": {
                    "type": "mesh_attribute",
                    # Mitsuba reads 'red','green','blue' as 'vertex_color' by default
                    # from face properties if they exist.
                    "name": "vertex_color",
                }
            }
        },
        # "emitter": { "type": "constant", "radiance": { "type": "rgb", "value": [1, 1, 1] } }
    }
    
    scene = mi.load_dict(scene_dict)
    image = mi.render(scene, spp=spp)
    
    return image

In [42]:
num_iterations = 50
B = np.zeros_like(E)

# radiosity_per_bounce = [E.copy()]

for i in tqdm(range(num_iterations), desc="Radiosity Bounces", leave=False):
    # Matrix multiplication computes the sum over j for all i simultaneously
    incoming_light = F_sparse @ B
    
    # Update radiosity for each color channel
    B = E + rho * incoming_light
    
    # # Clip to avoid negative values due to numerical precision issues
    # B = np.clip(B, 0, None)
    
    # radiosity_per_bounce.append(B.copy())

final_image = render_with_mitsuba(T, B, spp=16, smooth=False)
bitmap = mi.util.convert_to_bitmap(final_image)
bitmap.write(f"radiosity_final.png")

print("Radiosity calculation complete.")

Radiosity Bounces:   0%|          | 0/50 [00:00<?, ?it/s]

Radiosity calculation complete.
